# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,15.82,68,99,9.38,SH,1712194802
1,1,jabor,5.9210,169.6434,27.95,69,76,7.70,MH,1712194802
2,2,ribeira grande,38.5167,-28.7000,15.43,74,18,12.52,PT,1712194802
3,3,shortandy,51.7006,70.9994,1.65,76,6,2.97,KZ,1712194803
4,4,port-aux-francais,-49.3500,70.2167,5.58,65,12,9.87,TF,1712194803


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    size = "Humidity",
    color = "City"  
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df[(city_data_df['Humidity'] < 70) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] <= 25) & (city_data_df['Max Temp'] >=15)]

# Drop any rows with null values
narrow_df = narrow_df.dropna()
# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,66,cabo san lucas,22.8909,-109.9124,24.17,43,0,8.75,MX,1712194752
78,78,valparaiso,-33.0393,-71.6273,18.89,58,0,3.60,CL,1712194782
135,135,guerrero negro,27.9769,-114.0611,19.23,58,0,7.84,MX,1712194659
137,137,page,36.9147,-111.4558,20.06,18,0,2.24,US,1712194835
142,142,al jawf,29.5000,38.7500,19.77,24,0,4.55,SA,1712194836
183,183,mhamid,29.8200,-5.7200,21.28,19,0,2.87,MA,1712194847
252,252,san jose del cabo,23.0589,-109.6977,24.16,45,0,4.12,MX,1712194862
271,271,fort bragg,35.1390,-79.0060,20.75,29,0,9.26,US,1712194867
392,392,al muwayh,22.4333,41.7583,19.07,61,0,3.32,SA,1712194895
396,396,kayenta,36.7278,-110.2546,16.16,23,0,3.08,US,1712194896


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
66,cabo san lucas,MX,22.8909,-109.9124,43,
78,valparaiso,CL,-33.0393,-71.6273,58,
135,guerrero negro,MX,27.9769,-114.0611,58,
137,page,US,36.9147,-111.4558,18,
142,al jawf,SA,29.5000,38.7500,24,
183,mhamid,MA,29.8200,-5.7200,19,
252,san jose del cabo,MX,23.0589,-109.6977,45,
271,fort bragg,US,35.1390,-79.0060,29,
392,al muwayh,SA,22.4333,41.7583,61,
396,kayenta,US,36.7278,-110.2546,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit":20,
    'categories':'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
valparaiso - nearest hotel: Ibis Hotel
guerrero negro - nearest hotel: Plaza sal paraiso
page - nearest hotel: Clarion Inn Page - Lake Powell
al jawf - nearest hotel: No hotel found
mhamid - nearest hotel: Hotel Kasbah Azalay
san jose del cabo - nearest hotel: Hotel Posada Terranova
fort bragg - nearest hotel: Airborne Inn Lodging
al muwayh - nearest hotel: فندق ومطعم الموية
kayenta - nearest hotel: Wetherill Inn
galveston - nearest hotel: Comfort Inn & Suites Texas City
san patricio - nearest hotel: No hotel found
west hattiesburg - nearest hotel: Microtel Inn & Suites
alice springs - nearest hotel: Aurora Alice Springs
ibshaway - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
66,cabo san lucas,MX,22.8909,-109.9124,43,Comfort Rooms
78,valparaiso,CL,-33.0393,-71.6273,58,Ibis Hotel
135,guerrero negro,MX,27.9769,-114.0611,58,Plaza sal paraiso
137,page,US,36.9147,-111.4558,18,Clarion Inn Page - Lake Powell
142,al jawf,SA,29.5000,38.7500,24,No hotel found
183,mhamid,MA,29.8200,-5.7200,19,Hotel Kasbah Azalay
252,san jose del cabo,MX,23.0589,-109.6977,45,Hotel Posada Terranova
271,fort bragg,US,35.1390,-79.0060,29,Airborne Inn Lodging
392,al muwayh,SA,22.4333,41.7583,61,فندق ومطعم الموية
396,kayenta,US,36.7278,-110.2546,23,Wetherill Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600,
    size = "Humidity",
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name'] 
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)